# 🚀 Rocket Landing with REINFORCE

This notebook trains an AI agent to land a rocket on a planet using the REINFORCE algorithm.

## Setup

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q jax[cuda12] flax optax gymnax matplotlib numpy

In [ ]:
# Verify JAX is using GPU
import jax
print(f"JAX devices: {jax.devices()}")
print(f"Default backend: {jax.default_backend()}")

## Environment Definition

The rocket landing environment simulates a 2D orbital mechanics scenario.

In [ ]:
"""JAX implementation of rocket landing environment."""
from typing import Any, Optional, Tuple

import jax
import jax.numpy as jnp
from flax import struct
from gymnax.environments import environment, spaces

# GRAVITATIONAL_CONSTANT
G = 6.67430e-11


def angle_normalize(x: jax.Array) -> jax.Array:
    """Normalize the angle - radians."""
    return ((x + jnp.pi) % (2 * jnp.pi)) - jnp.pi


@struct.dataclass
class EnvState(environment.EnvState):
    """State of the rocket landing environment."""
    # position
    x: jnp.ndarray
    y: jnp.ndarray
    theta: jnp.ndarray

    # velocity
    dx: jnp.ndarray
    dy: jnp.ndarray
    omega: jnp.ndarray

    # engine
    throttle: jnp.ndarray
    gimbal: jnp.ndarray

    # fuel
    fuel: jnp.ndarray

    # target landing angle
    target_angle: jnp.ndarray

    # timestep
    time: jnp.ndarray


@struct.dataclass
class EnvParams(environment.EnvParams):
    """Parameters for the rocket landing environment."""
    max_steps_in_episode: int = 1500

    dt: float = 0.05

    # planet properties
    planet_radius: float = 50.0
    planet_mass: float = 5.0e14

    # rocket properties
    rocket_max_thrust: float = 38000.0
    rocket_max_gimbal: float = 0.3
    rocket_mass: float = 2600.0
    rocket_moment_of_inertia: float = 250.0
    rocket_initial_fuel: float = 5000.0
    rocket_fuel_consumption_rate: float = 100.

    # initialization parameters
    init_min_orbit_radius: float = 80.0
    init_max_orbit_radius: float = 150.0
    init_orbit_velocity_noise: float = 0.1

    # good landing thresholds
    landing_max_speed: float = 2.0
    landing_max_angle: float = 0.3
    landing_position_tolerance: float = jnp.pi/30
    landing_max_omega: float = 0.5

    # terminal rewards
    reward_landed: float = 100.0
    reward_sl_cp_co_ls: float = 1000.0
    reward_nsl_cp: float = 200.0
    reward_nsl_ncp: float = 100.0
    reward_sl_ncp_co_ls: float = 500.0
    reward_out_of_fuel: float = -50.0
    reward_out_of_bounds: float = -1000.0
    reward_timeout: float = -2000.0

    # non-terminal rewards
    reward_altitude_factor: float = 0.001
    reward_angular_position_factor: float = 0.0003
    reward_radial_velocity_factor: float = 0.0003
    reward_tangential_velocity_factor: float = 0.0005
    reward_angle_factor: float = 0.0003
    reward_angular_velocity_factor: float = 0.00005
    reward_fuel_penalty: float = 0.0  # Disabled to encourage engine use
    reward_low_velocity_near_surface: float = 0.0005
    reward_deceleration_with_fuel: float = 0.0003
    reward_retrograde_orientation: float = 0.0005
    reward_retrograde_burn: float = 0.02  # Reward for firing engine while aligned for braking


class RocketLander(environment.Environment[EnvState, EnvParams]):
    """JAX/Gymnax implementation of a 2D rocket landing environment."""

    def __init__(self):
        super().__init__()

    @property
    def default_params(self) -> EnvParams:
        return EnvParams()

    def step_env(
            self,
            key: jax.Array,
            state: EnvState,
            action: jax.Array,
            params: EnvParams,
    ) -> Tuple[jax.Array, EnvState, jax.Array, jax.Array, dict]:
        """Integrate"""

        ### deterministic step

        # gravity
        r = jnp.sqrt(state.x**2 + state.y**2)
        F_gravity_magnitude = G * params.planet_mass * params.rocket_mass / r**2
        # gravitational acceleration (a = F/m)
        a_gravity_magnitude = F_gravity_magnitude / params.rocket_mass

        a_gravity_magnitude = G * params.planet_mass / r**2

        a_g_x = -a_gravity_magnitude * state.x / r
        a_g_y = -a_gravity_magnitude * state.y / r

        ### control step

        # throttle and gimbal
        throttle_action = action[0]
        gimbal_action = action[1]

        throttle = jnp.clip(throttle_action, -1.0, 1.0)
        gimbal = jnp.clip(gimbal_action, -1.0, 1.0)

        throttle = (throttle + 1.0)/2.0
        gimbal *= params.rocket_max_gimbal

        # can only get as much throttle as we have fuel / consumption
        throttle = jnp.minimum(throttle, state.fuel / (params.rocket_fuel_consumption_rate * params.dt))

        # thrust
        thrust_angle = state.theta + gimbal
        F_thrust = throttle * params.rocket_max_thrust
        # acceleration (a = F/m)
        a_t_x = F_thrust * jnp.sin(thrust_angle) / params.rocket_mass
        a_t_y = F_thrust * jnp.cos(thrust_angle) / params.rocket_mass
        # torque
        F_torque = F_thrust * jnp.sin(gimbal)
        a_omega = F_torque / params.rocket_moment_of_inertia

        ### new state calculation
        # directional sum of gravitational and thrusticular accelerations
        a_x = a_g_x + a_t_x
        a_y = a_g_y + a_t_y

        dx = state.dx + a_x * params.dt
        dy = state.dy + a_y * params.dt
        omega = state.omega + a_omega * params.dt

        x = state.x + dx * params.dt
        y = state.y + dy * params.dt
        theta = state.theta + omega * params.dt

        fuel = state.fuel - throttle * params.rocket_fuel_consumption_rate * params.dt
        fuel = jnp.maximum(0.0, fuel)  # safety

        new_state = EnvState(
            x=x,
            y=y,
            theta=theta,
            dx=dx,
            dy=dy,
            omega=omega,
            throttle=throttle,
            gimbal=gimbal,
            fuel=fuel,
            target_angle=state.target_angle,
            time=state.time+params.dt
        )
        done = self.is_terminal(new_state, params)
        reward = self._compute_reward(state, new_state, params)
        key, noise_key = jax.random.split(key)
        obs = self.get_obs(new_state, params, noise_key)
        return obs, new_state, reward, done, {}

    def reset_env(
            self, key: jax.Array, params: EnvParams
    ) -> Tuple[jax.Array, EnvState]:
        """Reset environment by sampling."""
        key, angle_key, orbit_radius_key, orbit_vel_noise_key, target_angle_key = jax.random.split(key, 5)

        ### rocket
        # initial position and velocity
        angle_init = jax.random.uniform(angle_key, minval=0.0, maxval=2.0 * jnp.pi)

        orbit_radius_init = jax.random.uniform(orbit_radius_key, minval=params.init_min_orbit_radius, maxval=params.init_max_orbit_radius)

        x_init = orbit_radius_init * jnp.cos(angle_init)
        y_init = orbit_radius_init * jnp.sin(angle_init)

        orbit_velocity_magnitude_init = jnp.sqrt(G * params.planet_mass / orbit_radius_init)
        # Add noise scaled by the noise parameter
        velocity_noise = params.init_orbit_velocity_noise * (2.0 * jax.random.uniform(orbit_vel_noise_key) - 1.0)
        orbit_velocity_magnitude_init *= (1.0 + velocity_noise)

        dx_init = -orbit_velocity_magnitude_init * jnp.sin(angle_init)  # negative for counterclockwise orbit
        dy_init = orbit_velocity_magnitude_init * jnp.cos(angle_init)

        # initial angle and angular velocity
        theta_init = angle_init + (3/2)*jnp.pi  # rocket always orbiting "backwards" (thruster facing direction of motion)
        
        # Tidal locking: rocket rotates at the same rate as it orbits, keeping the same face toward the planet
        # For a counterclockwise orbit, the positional angle increases at rate: omega_orbit = v / r
        # The rocket must rotate at this same rate to maintain orientation relative to the planet
        omega_init = orbit_velocity_magnitude_init / orbit_radius_init

        target_angle = jax.random.uniform(target_angle_key, minval=0.0, maxval=2.0 * jnp.pi)

        state = EnvState(
            x=x_init,
            y=y_init,
            theta=theta_init,
            dx=dx_init,
            dy=dy_init,
            omega=omega_init,
            throttle=jnp.array(0.0),
            gimbal=jnp.array(0.0),
            fuel=jnp.array(params.rocket_initial_fuel),
            target_angle=target_angle,
            time=jnp.array(0)
        )

        return self.get_obs(state, params, key), state

    def get_obs(self, state: EnvState, params: EnvParams=None, key: jax.Array=None) -> jax.Array:
        """Return observation from state."""

        if params is None:  # safety - always provide params
            params = self.default_params

        r = jnp.sqrt(state.x ** 2 + state.y ** 2)
        altitude = r - params.planet_radius

        positional_angle = jnp.arctan2(state.y, state.x)

        delta_angle = state.target_angle - positional_angle
        delta_angle = angle_normalize(delta_angle)

        radial_vel = (state.x * state.dx + state.y * state.dy) / r
        tangential_vel = (state.x * state.dy - state.y * state.dx) / r

        theta_relative = state.theta - positional_angle
        theta_relative = angle_normalize(theta_relative)

        # observation
        obs = jnp.array(
            [
                # polar coordinates (position)
                altitude,
                delta_angle,
                # polar coordinates (velocity)
                radial_vel,
                tangential_vel,
                # rotational values
                theta_relative,
                state.omega,
                # engine
                state.throttle,
                state.gimbal,
                state.fuel
            ]
        )

        return obs

    def is_terminal(self, state: EnvState, params: EnvParams) -> jax.Array:
        """Check whether state is terminal."""

        # timeout based on time in seconds (max_steps * dt)
        max_time = params.max_steps_in_episode * params.dt
        timeout = state.time >= max_time

        r = jnp.sqrt(state.x ** 2 + state.y ** 2)

        landed = r <= params.planet_radius

        escaped = r > params.planet_radius * 4

        done = landed | timeout | escaped
        return jnp.array(done)

    def _compute_reward(self, old_state: EnvState, new_state: EnvState, params: EnvParams) -> jax.Array:
        """Compute reward."""
        
        ### preliminary calculations

        ## new state
        r = jnp.sqrt(new_state.x ** 2 + new_state.y ** 2)
        altitude = r - params.planet_radius
        normalized_altitude = jnp.clip(altitude / (params.init_max_orbit_radius - params.planet_radius), 0.0, 2.0)

        # terminal conditions
        landed = r <= params.planet_radius
        escaped = r > params.planet_radius * 4
        max_time = params.max_steps_in_episode * params.dt
        timeout = new_state.time >= max_time
        is_terminal = landed | escaped | timeout

        # angular distance from target landing spot
        positional_angle = jnp.arctan2(new_state.y, new_state.x)
        delta_angle = jnp.abs(angle_normalize(new_state.target_angle - positional_angle))

        # various velocities
        v_total = jnp.sqrt(new_state.dx ** 2 + new_state.dy ** 2)
        v_radial = (new_state.x * new_state.dx + new_state.y * new_state.dy) / r
        v_tangential = (new_state.x * new_state.dy - new_state.y * new_state.dx) / r

        # get rocket orientation relative to surface
        theta_relative = angle_normalize(new_state.theta - positional_angle)

        ## old state (same calculations)
        r_old = jnp.sqrt(old_state.x ** 2 + old_state.y ** 2)
        altitude_old = r_old - params.planet_radius
        normalized_altitude_old = jnp.clip(altitude_old / (params.init_max_orbit_radius - params.planet_radius), 0.0, 2.0)

        positional_angle_old = jnp.arctan2(old_state.y, old_state.x)
        delta_angle_old = jnp.abs(angle_normalize(old_state.target_angle - positional_angle_old))

        v_total_old = jnp.sqrt(old_state.dx ** 2 + old_state.dy ** 2)
        v_radial_old = (old_state.x * old_state.dx + old_state.y * old_state.dy) / r_old
        v_tangential_old = (old_state.x * old_state.dy - old_state.y * old_state.dx) / r_old

        theta_relative_old = jnp.abs(angle_normalize(old_state.theta - positional_angle_old))

        ## landing conditions
        slow = v_total < params.landing_max_speed
        correct_position = delta_angle <= params.landing_position_tolerance
        correct_orientation = jnp.abs(theta_relative) <= params.landing_max_angle
        low_spin = jnp.abs(new_state.omega) <= params.landing_max_omega

        ### terminal rewards

        terminal_reward = jnp.array(0.0)

        # ideal landing: slow, correct position, correct orientation, low spin
        sl_cp_co_ls = landed & slow & correct_position & correct_orientation & low_spin
        terminal_reward = jnp.where(sl_cp_co_ls, params.reward_sl_cp_co_ls, terminal_reward)

        # ideal landing at incorrect position
        sl_ncp_co_ls = landed & slow & ~correct_position & correct_orientation & low_spin
        terminal_reward = jnp.where(sl_ncp_co_ls, params.reward_sl_ncp_co_ls * jnp.exp(-delta_angle * 2.0), terminal_reward)

        # slow landing but wrong orientation/spin
        sl_nco_nls = landed & slow & ~(correct_orientation & low_spin)
        slow_reward = 100.0 * jnp.exp(-delta_angle)
        terminal_reward = jnp.where(sl_nco_nls, slow_reward, terminal_reward)

        # non-ideal (fast) landing at correct position
        nsl_cp = landed & ~slow & correct_position
        terminal_reward = jnp.where(nsl_cp, params.reward_nsl_cp, terminal_reward)

        # non-ideal (fast) landing at incorrect position
        nsl_ncp = landed & ~slow & ~correct_position
        terminal_reward = jnp.where(nsl_ncp, params.reward_nsl_ncp, terminal_reward)

        # escaped bounds
        terminal_reward = jnp.where(escaped, params.reward_out_of_bounds, terminal_reward)

        # timeout - heavily penalized to discourage staying in orbit
        terminal_reward = jnp.where(timeout & ~landed & ~escaped, params.reward_timeout, terminal_reward)

        ### non-terminal rewards
        
        shaping_reward = jnp.array(0.0)

        # reward retrograde burn
        thrust_angle = new_state.theta + new_state.gimbal
        velocity_angle = jnp.arctan2(new_state.dy, new_state.dx)
        alignment = jnp.cos(thrust_angle - velocity_angle)
        retrograde_alignment = (1.0 - alignment) / 2.0
        throttle_used = new_state.throttle > 0.1
        velocity_factor = jnp.minimum(v_total / 10.0, 1.0)
        retrograde_burn_reward = params.reward_retrograde_burn * retrograde_alignment * throttle_used * velocity_factor
        shaping_reward += retrograde_burn_reward

        # reward tangential velocity -> 0
        tangential_velocity_change = jnp.abs(v_tangential) - jnp.abs(v_tangential_old)
        shaping_reward += -0.3 * tangential_velocity_change

        # if delta_angle -> 0, reward altitude -> 0
        delta_angle_factor = jnp.exp(-delta_angle * 3)
        altitude_change = normalized_altitude - normalized_altitude_old
        shaping_reward += -1.0 * delta_angle_factor * altitude_change

        # if landing (low altitude and low velocity), reward theta_relative -> 0
        near_landing_factor = jnp.exp(-3.0 * normalized_altitude) * jnp.exp(-v_total)
        theta_relative_change = jnp.abs(theta_relative) - jnp.abs(theta_relative_old)
        shaping_reward += -1.0 * near_landing_factor * theta_relative_change

        # if landing (same), reward angular_velocity -> 0
        omega_change = jnp.abs(new_state.omega) - jnp.abs(old_state.omega)
        shaping_reward += -0.1 * near_landing_factor * omega_change

        # reward being near target (altitude -> 0, delta_angle -> 0)
        low_altitude_factor = jnp.exp(-normalized_altitude * 2.0)
        near_target_factor = jnp.exp(-delta_angle * 2.0)
        landing_zone_bonus = 0.1 * low_altitude_factor * near_target_factor * near_target_factor
        shaping_reward += landing_zone_bonus

        # reward velocity -> 0 (only reward decrease, don't punish increase)
        velocity_change = v_total - v_total_old
        shaping_reward += -0.1 * jnp.maximum(0.0, -velocity_change)

        # punish high angular velocity
        high_omega = jnp.abs(new_state.omega) > 2.0
        angular_velocity_punishment = 0.1 * high_omega * (jnp.abs(new_state.omega) - 2.0)
        shaping_reward -= angular_velocity_punishment

        # punish (angular) distance from target
        distance_from_target_penalty = 0.02 * delta_angle
        shaping_reward -= distance_from_target_penalty

        # FINAL REWARD: Terminal rewards REPLACE shaping, not add
        reward = jnp.where(is_terminal, terminal_reward, shaping_reward)

        return reward

    @property
    def name(self) -> str:
        """Environment name."""
        return "RocketLander"

    @property
    def num_actions(self) -> int:
        """Number of actions possible in environment."""
        return 2

    def action_space(self, params: Optional[EnvParams] = None) -> spaces.Box:
        """Action space of the environment."""
        return spaces.Box(
            low=-1.0,
            high=1.0,
            shape=(2,),
            dtype=jnp.float32,
        )

    def observation_space(self, params: Optional[EnvParams] = None) -> spaces.Box:
        """Observation space of the environment."""
        # [altitude, delta_angle, radial_vel, tangential_vel, theta_relative, omega, throttle, gimbal, fuel]
        low = jnp.array([-10.0, -jnp.pi, -50.0, -50.0, -jnp.pi, -10.0, 0.0, -0.5, 0.0])
        high = jnp.array([150.0, jnp.pi, 50.0, 50.0, jnp.pi, 10.0, 1.0, 0.5, 5000.0])
        return spaces.Box(low, high, shape=(9,), dtype=jnp.float32)


# Create environment instance
env = RocketLander()
env_params = env.default_params
print("Environment created successfully!")
print(f"  Observation space: {env.observation_space().shape}")
print(f"  Action space: {env.action_space().shape}")

## Visualization Utilities

In [ ]:
"""Visualization utilities for the rocket landing environment."""
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection


def visualize_trajectory(
    states: list,
    params: EnvParams,
    title: str = "Rocket Landing Trajectory",
    save_path: str = None
):
    """
    Visualize the rocket trajectory around the planet.

    Args:
        states: List of EnvState objects from an episode
        params: Environment parameters
        title: Plot title
        save_path: If provided, save figure to this path
    """
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))

    # Draw planet
    theta = np.linspace(0, 2 * np.pi, 100)
    planet_x = params.planet_radius * np.cos(theta)
    planet_y = params.planet_radius * np.sin(theta)
    ax.fill(planet_x, planet_y, color='#3d5a80', alpha=0.8, label='Planet')
    ax.plot(planet_x, planet_y, color='#1d3557', linewidth=2)

    # Extract trajectory
    xs = np.array([float(s.x) for s in states])
    ys = np.array([float(s.y) for s in states])
    throttles = np.array([float(s.throttle) for s in states])

    # Draw trajectory colored by throttle
    points = np.array([xs, ys]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    norm = plt.Normalize(0, 1)
    lc = LineCollection(segments, cmap='YlOrRd', norm=norm, linewidth=2, alpha=0.8)
    lc.set_array(throttles[:-1])
    ax.add_collection(lc)
    cbar = plt.colorbar(lc, ax=ax, label='Throttle')
    cbar.ax.yaxis.set_tick_params(color='white')
    cbar.ax.yaxis.label.set_color('white')
    plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='white')

    # Mark start and end
    ax.scatter(xs[0], ys[0], s=100, c='green', marker='o', zorder=5, label='Start')
    ax.scatter(xs[-1], ys[-1], s=100, c='red', marker='x', zorder=5, label='End')

    # Draw target landing site
    target_angle = float(states[0].target_angle)
    target_x = params.planet_radius * np.cos(target_angle)
    target_y = params.planet_radius * np.sin(target_angle)
    ax.scatter(target_x, target_y, s=200, c='yellow', marker='*', zorder=5,
               edgecolors='black', label='Target')

    # Draw rocket orientation arrows
    n_arrows = min(30, len(states))
    arrow_indices = np.linspace(0, len(states) - 1, n_arrows, dtype=int)
    for idx in arrow_indices:
        s = states[idx]
        x, y = float(s.x), float(s.y)
        theta = float(s.theta)
        dx = np.sin(theta) * 8
        dy = np.cos(theta) * 8
        ax.arrow(x, y, dx, dy, head_width=4, head_length=2, fc='#cccccc', ec='white', alpha=0.9)

    # Set field of view
    fov = params.init_max_orbit_radius * 1.2
    ax.set_xlim(-fov, fov)
    ax.set_ylim(-fov, fov)
    ax.set_aspect('equal')
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.set_title(title)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

    # Dark background for space
    ax.set_facecolor('#0d1b2a')
    fig.patch.set_facecolor('#0d1b2a')
    ax.tick_params(colors='white')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.title.set_color('white')
    ax.legend(facecolor='#1b263b', edgecolor='white', labelcolor='white')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, facecolor=fig.get_facecolor())
        plt.close()
    else:
        plt.show()

    return fig


print("Visualization utilities loaded!")

## Neural Network Policy and Training

In [ ]:
import pickle
import jax
import jax.numpy as jnp
import jax.random as jr
from jax.random import PRNGKey
import optax


# =============================================================================
# Neural Network
# =============================================================================

def initialize_mlp(layer_sizes, key: PRNGKey, scale: float = 1e-2):
    """
    Inputs:
        layer_sizes (tuple) Tuple of shapes of the neural network layers.
        key (PRNGKey)
        scale (float) standard deviation of initial weights and biases

    Return:
        params (List) Tuple of weights and biases
    """

    keys = jr.split(key, 2 * len(layer_sizes))
    params = []

    for i in range(len(layer_sizes) - 1):
        input_size, output_size = layer_sizes[i], layer_sizes[i + 1]
        W = jr.normal(keys[2 * i], (input_size, output_size)) * scale
        b = jr.normal(keys[2 * i + 1], (output_size,)) * scale
        params.append((W, b))

    return params


def policy(params, x):
    """
    Gaussian policy network.
    
    Inputs:
        params (list): Network parameters as list of (W, b) tuples.
        x (array): Input observation of shape (obs_dim,).
        
    Returns:
        mean (array): Mean of Gaussian for each action dimension.
        log_std (array): Log standard deviation for each action dimension.
    """
    # Shared layers
    for (W, b) in params[:-1]:
        x = jnp.dot(x, W) + b
        x = jax.nn.relu(x)

    # Output layer: first half is mean, second half is log_std
    W, b = params[-1]
    output = jnp.dot(x, W) + b
    
    action_dim = output.shape[-1] // 2
    mean = jnp.tanh(output[:action_dim])  # Bound mean to [-1, 1]
    log_std = output[action_dim:]
    log_std = jnp.clip(log_std, -2.0, 0.5)  # Bound std to reasonable range
    
    return mean, log_std


def get_action(params, x, key: PRNGKey):
    """
    Sample an action from the Gaussian policy.

    Inputs:
        params (list): Network parameters.
        x (array): Input observation.
        key (PRNGKey): Random key for sampling.
     
    Returns:
        action (array): Sampled action, clipped to [-1, 1].
        mean (array): Mean of the Gaussian.
        log_std (array): Log standard deviation.
    """
    mean, log_std = policy(params, x)
    std = jnp.exp(log_std)
    noise = jr.normal(key, shape=mean.shape)
    action = mean + std * noise
    action = jnp.clip(action, -1.0, 1.0)  # Clip to valid action range
    return action, mean, log_std


def get_log_prob(params, x, action):
    """
    Compute log probability of action under the Gaussian policy.

    Inputs:
        params (list): Network parameters.
        x (array): Input observation.
        action (array): Action taken.
     
    Returns:
        log_prob (float): Log probability of the action.
    """
    mean, log_std = policy(params, x)
    std = jnp.exp(log_std)
    
    # Gaussian log probability
    log_prob = -0.5 * jnp.sum(
        ((action - mean) / std) ** 2 + 2 * log_std + jnp.log(2 * jnp.pi)
    )
    return log_prob


@jax.jit
def update_delta(delta, grad_theta):
    """
    Update the parameter gradients delta with new gradient.
    """
    updated_delta = jax.tree.map(lambda x, y: x + y, delta, grad_theta)
    return updated_delta, None


# =============================================================================
# Rollout Function
# =============================================================================

def rollout(params, env_params, rng_input: PRNGKey, steps_in_episode: int):
    """
    Rollout an episode using the policy.
    
    Inputs:
        params: Network parameters.
        env_params: Environment parameters.
        rng_input: Random key.
        steps_in_episode: Number of steps to run.
        
    Returns:
        Tuple of (obs, state, action, reward, next_obs, done) arrays.
    """
    rng_reset, rng_episode = jr.split(rng_input)
    obs, state = env.reset_env(rng_reset, env_params)

    def policy_step(state_input, tmp):
        """Single step of the policy in the environment."""
        obs, state, rng = state_input
        rng, rng_action, rng_step = jr.split(rng, 3)
        
        action, mean, log_std = get_action(params, obs, rng_action)
        next_obs, next_state, reward, done, _ = env.step_env(
            rng_step, state, action, env_params
        )
        
        carry = [next_obs, next_state, rng]
        return carry, [obs, state, action, reward, next_obs, done]

    # Scan over episode steps
    _, scan_out = jax.lax.scan(
        policy_step,
        [obs, state, rng_episode],
        (),
        length=steps_in_episode,
    )
    
    return scan_out


jit_rollout = jax.jit(rollout, static_argnums=3)


# =============================================================================
# REINFORCE Loss
# =============================================================================

def loss_REINFORCE(params, obs, action, reward, baseline, gamma: float = 0.99):
    """
    Compute REINFORCE loss with baseline.

    Inputs:
        params: Network parameters.
        obs (array): Batch of observations (batch, time, obs_dim).
        action (array): Batch of actions (batch, time, action_dim).
        reward (array): Batch of rewards (batch, time).
        baseline (array): Baseline values (time,).
        gamma (float): Discount factor.

    Returns:
        delta: Accumulated gradients.
        Gt (array): Discounted returns.
    """

    def trajectory_gradients(reward, obs, action, baseline, delta):
        G_init = 0.0

        def step(carry, variables):
            G, delta = carry
            r, obs, action, b = variables

            # Calculate discounted return and advantage
            G = gamma * G + r
            advantage = G - b

            # Compute gradient of log probability
            def neg_log_prob(params):
                return -get_log_prob(params, obs, action)

            grad_delta = jax.grad(neg_log_prob)(params)
            grad_delta = jax.tree.map(lambda gd: gd * advantage, grad_delta)
            delta, _ = update_delta(delta, grad_delta)

            return (G, delta), G

        # Iterate backwards in time
        variables = (reward[::-1], obs[::-1], action[::-1], baseline[::-1])
        (_, delta), Gt = jax.lax.scan(step, (G_init, delta), variables)
        
        return delta, Gt

    # Parallelize over batch
    parallel_trajectory_gradients = jax.vmap(
        trajectory_gradients, in_axes=(0, 0, 0, None, None)
    )
    
    # Initialize delta to zeros
    delta = jax.tree.map(lambda t: jnp.zeros(t.shape), params)

    # Compute gradients in parallel and sum
    deltas, Gs = parallel_trajectory_gradients(reward, obs, action, baseline, delta)
    delta, _ = jax.lax.scan(update_delta, delta, deltas)

    return delta, jnp.array(Gs)


loss_REINFORCE = jax.jit(loss_REINFORCE)


# =============================================================================
# Baseline Functions
# =============================================================================

def mean_baseline(Gs):
    """Compute constant baseline as mean of discounted returns."""
    T = Gs.shape[1]
    mean_reward = jnp.mean(Gs)
    return jnp.ones((T,)) * mean_reward


def timedependent_baseline(Gs):
    """Compute time-dependent baseline."""
    mean_reward = jnp.mean(Gs, axis=0)  # Mean over batches
    cumulative_rewards = jnp.cumsum(mean_reward)
    baseline = cumulative_rewards / jnp.arange(1, Gs.shape[1] + 1)
    return mean_reward - baseline


print("Training utilities loaded!")

## Training Function

In [ ]:
def visualize_policy(params, key, title="Policy Trajectory", save_path=None):
    """Run a single episode and visualize the trajectory."""
    # Run rollout
    obs, state, action, reward, next_obs, done = rollout(
        params, env_params, key, env_params.max_steps_in_episode
    )
    
    # Convert states to list for visualization
    states = []
    for i in range(len(state.x)):
        s = EnvState(
            x=state.x[i],
            y=state.y[i],
            theta=state.theta[i],
            dx=state.dx[i],
            dy=state.dy[i],
            omega=state.omega[i],
            throttle=state.throttle[i],
            gimbal=state.gimbal[i],
            fuel=state.fuel[i],
            target_angle=state.target_angle[i],
            time=state.time[i],
        )
        states.append(s)
        
        # Stop if done
        if done[i]:
            break
    
    # Only sum rewards up to the terminal step (after that, rewards are garbage)
    episode_length = len(states)
    total_reward = float(jnp.sum(reward[:episode_length]))
    print(f"  Total reward: {total_reward:.2f}")
    print(f"  Episode length: {episode_length} steps")
    
    visualize_trajectory(states, env_params, title=f"{title} (R={total_reward:.1f})", save_path=save_path)


def train(
    num_iters: int = 5000,
    steps_in_episode: int = 2000,
    lr: float = 0.001,
    gamma: float = 0.99,
    n_batches: int = 32,
    hidden_size: int = 128,
    n_hidden_layers: int = 2,
    visualize_every: int = 500,
    seed: int = 42,
    grad_clip: float = 1.0,
):
    """
    Train the rocket landing policy using REINFORCE.
    
    Args:
        num_iters: Number of training iterations.
        steps_in_episode: Steps per episode.
        lr: Learning rate.
        gamma: Discount factor.
        n_batches: Number of parallel episodes per iteration.
        hidden_size: Size of hidden layers.
        n_hidden_layers: Number of hidden layers.
        visualize_every: Visualize trajectory every n iterations.
        seed: Random seed.
        grad_clip: Maximum gradient norm for clipping.
    """
    key = PRNGKey(seed)
    
    # Network architecture: obs_dim -> hidden layers -> 2*action_dim (mean + log_std)
    obs_dim = 9  # From rocket_env observation space
    action_dim = 2  # throttle and gimbal
    hidden_layers = tuple([hidden_size] * n_hidden_layers)
    layer_sizes = (obs_dim,) + hidden_layers + (2 * action_dim,)
    
    # Initialize network and optimizer
    key, subkey = jr.split(key)
    params = initialize_mlp(layer_sizes, key=subkey)
    
    # Adam optimizer with gradient clipping for stability
    optim = optax.chain(
        optax.clip_by_global_norm(grad_clip),
        optax.adam(learning_rate=lr),
    )
    opt_state = optim.init(params)
    
    # Initialize baseline storage
    Gs = jnp.zeros((n_batches, steps_in_episode))
    
    # Training keys
    key, subkey = jr.split(key)
    iter_keys = jr.split(subkey, num_iters)
    
    # Parallel rollout function
    parallel_rollout = jax.vmap(rollout, in_axes=(None, None, 0, None))
    
    # Training step function
    def step(carry, key):
        params, opt_state, Gs = carry
        
        # Generate rollouts
        keys = jr.split(key, n_batches)
        obs, state, action, reward, next_obs, done = parallel_rollout(
            params, env_params, keys, steps_in_episode
        )
        
        # Compute baseline from previous returns
        baseline = mean_baseline(Gs)
        
        # Compute gradients
        delta, Gs_new = loss_REINFORCE(params, obs, action, reward, baseline, gamma)
        
        # Update parameters
        updates, opt_state = optim.update(delta, opt_state, params)
        new_params = optax.apply_updates(params, updates)
        
        # Mean episode reward for logging
        mean_reward = jnp.mean(jnp.sum(reward, axis=-1))
        
        carry = (new_params, opt_state, Gs_new)
        return carry, mean_reward

    # Compile the step function
    step = jax.jit(step)
    
    # Training loop with periodic visualization
    history = []
    current_params = params
    current_opt_state = opt_state
    current_Gs = Gs
    
    print(f"Starting training for {num_iters} iterations...")
    print(f"  Network architecture: {layer_sizes}")
    print(f"  Episodes per iteration: {n_batches}")
    print(f"  Steps per episode: {steps_in_episode}")
    print(f"  Learning rate: {lr}")
    print(f"  Discount factor: {gamma}")
    print(f"  Gradient clipping: {grad_clip}")
    print()
    
    for i in range(num_iters):
        # Training step
        (current_params, current_opt_state, current_Gs), mean_reward = step(
            (current_params, current_opt_state, current_Gs), iter_keys[i]
        )
        history.append(float(mean_reward))
        
        # Logging
        if (i + 1) % 100 == 0:
            avg_reward = jnp.mean(jnp.array(history[-100:]))
            print(f"Iteration {i + 1}/{num_iters} | Avg Reward (last 100): {avg_reward:.2f}")
        
        # Visualization
        if (i + 1) % visualize_every == 0:
            print(f"\nVisualizing policy at iteration {i + 1}...")
            key, vis_key = jr.split(key)
            visualize_policy(current_params, vis_key, title=f"Iteration {i + 1}")
    
    # Final visualization
    print(f"\nTraining complete! Final policy visualization...")
    key, vis_key = jr.split(key)
    visualize_policy(current_params, vis_key, title="Final Policy")
    
    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.plot(history, alpha=0.3, label='Episode Reward')
    # Smoothed curve
    window = min(100, len(history) // 10)
    if window > 1:
        smoothed = jnp.convolve(jnp.array(history), jnp.ones(window) / window, mode='valid')
        plt.plot(range(window - 1, len(history)), smoothed, label=f'Smoothed (window={window})')
    plt.xlabel('Iteration')
    plt.ylabel('Mean Episode Reward')
    plt.title('Training Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return current_params, history


print("Training function ready!")

## Run Training

Adjust hyperparameters as needed. The default values are optimized for GPU training.

In [ ]:
# Train the agent
trained_params, history = train(
    num_iters=50000,
    steps_in_episode=1500,
    lr=0.0002,
    gamma=0.995,
    n_batches=128,
    hidden_size=256,
    n_hidden_layers=3,
    visualize_every=500,
    seed=42,
    grad_clip=0.5,
)

## Test Trained Policy

Run multiple test episodes to see how the trained agent performs.

In [ ]:
# Test the trained policy with multiple episodes
key = PRNGKey(123)
for i in range(5):
    key, subkey = jr.split(key)
    print(f"\n=== Test Episode {i+1} ===")
    visualize_policy(trained_params, subkey, title=f"Test Episode {i+1}")

## Save/Load Model

In [ ]:
# Save trained parameters
with open('trained_rocket_lander.pkl', 'wb') as f:
    pickle.dump(trained_params, f)
print("Model saved to 'trained_rocket_lander.pkl'")

In [ ]:
# Load trained parameters
with open('trained_rocket_lander.pkl', 'rb') as f:
    loaded_params = pickle.load(f)
print("Model loaded from 'trained_rocket_lander.pkl'")

# Test loaded model
key = PRNGKey(456)
visualize_policy(loaded_params, key, title="Loaded Model Test")

# 🚀 Rocket Landing with REINFORCE

This notebook trains a neural network policy to land a rocket on a planet using the REINFORCE algorithm with a Gaussian policy.

**Features:**
- JAX-based physics simulation
- Vectorized environment for fast training
- Gaussian policy for continuous control
- Reward shaping to guide learning

## 1. Setup

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q jax[cuda12] flax optax gymnax matplotlib numpy

In [ ]:
# Import libraries
import pickle
from typing import Any, Optional, Tuple

import jax
import jax.numpy as jnp
import jax.random as jr
from jax.random import PRNGKey
import optax
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection
from flax import struct
from gymnax.environments import environment, spaces

# Verify JAX is using GPU
print(f"JAX version: {jax.__version__}")
print(f"JAX backend: {jax.default_backend()}")
print(f"JAX devices: {jax.devices()}")

## 2. Environment Definition

In [ ]:
# Gravitational constant
G = 6.67430e-11


def angle_normalize(x: jax.Array) -> jax.Array:
    """Normalize the angle - radians."""
    return ((x + jnp.pi) % (2 * jnp.pi)) - jnp.pi


@struct.dataclass
class EnvState(environment.EnvState):
    """State of the rocket landing environment."""
    # position
    x: jnp.ndarray
    y: jnp.ndarray
    theta: jnp.ndarray

    # velocity
    dx: jnp.ndarray
    dy: jnp.ndarray
    omega: jnp.ndarray

    # engine
    throttle: jnp.ndarray
    gimbal: jnp.ndarray

    # fuel
    fuel: jnp.ndarray

    # target landing angle
    target_angle: jnp.ndarray

    # timestep
    time: jnp.ndarray


@struct.dataclass
class EnvParams(environment.EnvParams):
    """Parameters for the rocket landing environment."""
    max_steps_in_episode: int = 1500
    dt: float = 0.05

    # planet properties
    planet_radius: float = 50.0
    planet_mass: float = 5.0e14

    # rocket properties
    rocket_max_thrust: float = 38000.0
    rocket_max_gimbal: float = 0.3
    rocket_mass: float = 2600.0
    rocket_moment_of_inertia: float = 250.0
    rocket_initial_fuel: float = 5000.0
    rocket_fuel_consumption_rate: float = 100.

    # initialization parameters
    init_min_orbit_radius: float = 80.0
    init_max_orbit_radius: float = 150.0
    init_orbit_velocity_noise: float = 0.1

    # good landing thresholds
    landing_max_speed: float = 2.0
    landing_max_angle: float = 0.3
    landing_max_omega: float = 0.5
    landing_position_tolerance: float = jnp.pi/30

    # terminal rewards
    reward_landed: float = 100.0
    reward_sl_cp_co_ls: float = 1000.0
    reward_nsl_cp: float = 200.0
    reward_nsl_ncp: float = 100.0
    reward_sl_ncp_co_ls: float = 500.0
    reward_out_of_fuel: float = -50.0
    reward_out_of_bounds: float = -1000.0
    reward_timeout: float = -2000.0

    # non-terminal rewards
    reward_retrograde_burn: float = 0.02

In [ ]:
class RocketLander(environment.Environment[EnvState, EnvParams]):
    """JAX/Gymnax implementation of a 2D rocket landing environment."""

    def __init__(self):
        super().__init__()

    @property
    def default_params(self) -> EnvParams:
        return EnvParams()

    def step_env(self, key: jax.Array, state: EnvState, action: jax.Array, params: EnvParams):
        """Integrate physics for one timestep."""
        # Gravity
        r = jnp.sqrt(state.x**2 + state.y**2)
        a_gravity_magnitude = G * params.planet_mass / r**2
        a_g_x = -a_gravity_magnitude * state.x / r
        a_g_y = -a_gravity_magnitude * state.y / r

        # Throttle and gimbal control
        throttle = jnp.clip(action[0], -1.0, 1.0)
        gimbal = jnp.clip(action[1], -1.0, 1.0)
        throttle = (throttle + 1.0)/2.0
        gimbal *= params.rocket_max_gimbal
        throttle = jnp.minimum(throttle, state.fuel / (params.rocket_fuel_consumption_rate * params.dt))

        # Thrust
        thrust_angle = state.theta + gimbal
        F_thrust = throttle * params.rocket_max_thrust
        a_t_x = F_thrust * jnp.sin(thrust_angle) / params.rocket_mass
        a_t_y = F_thrust * jnp.cos(thrust_angle) / params.rocket_mass
        F_torque = F_thrust * jnp.sin(gimbal)
        a_omega = F_torque / params.rocket_moment_of_inertia

        # Physics integration
        a_x, a_y = a_g_x + a_t_x, a_g_y + a_t_y
        dx = state.dx + a_x * params.dt
        dy = state.dy + a_y * params.dt
        omega = state.omega + a_omega * params.dt
        x = state.x + dx * params.dt
        y = state.y + dy * params.dt
        theta = state.theta + omega * params.dt
        fuel = jnp.maximum(0.0, state.fuel - throttle * params.rocket_fuel_consumption_rate * params.dt)

        new_state = EnvState(x=x, y=y, theta=theta, dx=dx, dy=dy, omega=omega,
                             throttle=throttle, gimbal=gimbal, fuel=fuel,
                             target_angle=state.target_angle, time=state.time+params.dt)
        
        done = self.is_terminal(new_state, params)
        reward = self._compute_reward(state, new_state, params)
        key, noise_key = jax.random.split(key)
        obs = self.get_obs(new_state, params, noise_key)
        return obs, new_state, reward, done, {}

    def reset_env(self, key: jax.Array, params: EnvParams):
        """Reset environment by sampling initial orbit."""
        key, angle_key, orbit_radius_key, orbit_vel_noise_key, target_angle_key = jax.random.split(key, 5)
        
        angle_init = jax.random.uniform(angle_key, minval=0.0, maxval=2.0 * jnp.pi)
        orbit_radius_init = jax.random.uniform(orbit_radius_key, minval=params.init_min_orbit_radius, maxval=params.init_max_orbit_radius)
        
        x_init = orbit_radius_init * jnp.cos(angle_init)
        y_init = orbit_radius_init * jnp.sin(angle_init)
        
        orbit_velocity_magnitude_init = jnp.sqrt(G * params.planet_mass / orbit_radius_init)
        velocity_noise = params.init_orbit_velocity_noise * (2.0 * jax.random.uniform(orbit_vel_noise_key) - 1.0)
        orbit_velocity_magnitude_init *= (1.0 + velocity_noise)
        
        dx_init = -orbit_velocity_magnitude_init * jnp.sin(angle_init)
        dy_init = orbit_velocity_magnitude_init * jnp.cos(angle_init)
        theta_init = angle_init + (3/2)*jnp.pi
        target_angle = jax.random.uniform(target_angle_key, minval=0.0, maxval=2.0 * jnp.pi)

        state = EnvState(x=x_init, y=y_init, theta=theta_init, dx=dx_init, dy=dy_init,
                         omega=jnp.array(0.0), throttle=jnp.array(0.0), gimbal=jnp.array(0.0),
                         fuel=jnp.array(params.rocket_initial_fuel), target_angle=target_angle, time=jnp.array(0))
        return self.get_obs(state, params, key), state

    def get_obs(self, state: EnvState, params: EnvParams=None, key: jax.Array=None) -> jax.Array:
        """Get observation from state."""
        if params is None:
            params = self.default_params
        r = jnp.sqrt(state.x ** 2 + state.y ** 2)
        altitude = r - params.planet_radius
        positional_angle = jnp.arctan2(state.y, state.x)
        delta_angle = angle_normalize(state.target_angle - positional_angle)
        radial_vel = (state.x * state.dx + state.y * state.dy) / r
        tangential_vel = (state.x * state.dy - state.y * state.dx) / r
        theta_relative = angle_normalize(state.theta - positional_angle)
        return jnp.array([altitude, delta_angle, radial_vel, tangential_vel,
                          theta_relative, state.omega, state.throttle, state.gimbal, state.fuel])

    def is_terminal(self, state: EnvState, params: EnvParams) -> jax.Array:
        """Check whether state is terminal."""
        max_time = params.max_steps_in_episode * params.dt
        timeout = state.time >= max_time
        r = jnp.sqrt(state.x ** 2 + state.y ** 2)
        landed = r <= params.planet_radius
        escaped = r > params.planet_radius * 4
        return jnp.array(landed | timeout | escaped)

    def _compute_reward(self, old_state: EnvState, new_state: EnvState, params: EnvParams) -> jax.Array:
        """Compute reward with shaping."""
        # New state calculations
        r = jnp.sqrt(new_state.x ** 2 + new_state.y ** 2)
        altitude = r - params.planet_radius
        normalized_altitude = jnp.clip(altitude / (params.init_max_orbit_radius - params.planet_radius), 0.0, 2.0)

        landed = r <= params.planet_radius
        escaped = r > params.planet_radius * 4
        max_time = params.max_steps_in_episode * params.dt
        timeout = new_state.time >= max_time
        is_terminal = landed | escaped | timeout

        positional_angle = jnp.arctan2(new_state.y, new_state.x)
        delta_angle = jnp.abs(angle_normalize(new_state.target_angle - positional_angle))
        v_total = jnp.sqrt(new_state.dx ** 2 + new_state.dy ** 2)
        v_tangential = (new_state.x * new_state.dy - new_state.y * new_state.dx) / r
        theta_relative = angle_normalize(new_state.theta - positional_angle)

        # Old state calculations
        r_old = jnp.sqrt(old_state.x ** 2 + old_state.y ** 2)
        normalized_altitude_old = jnp.clip((r_old - params.planet_radius) / (params.init_max_orbit_radius - params.planet_radius), 0.0, 2.0)
        positional_angle_old = jnp.arctan2(old_state.y, old_state.x)
        v_total_old = jnp.sqrt(old_state.dx ** 2 + old_state.dy ** 2)
        v_tangential_old = (old_state.x * old_state.dy - old_state.y * old_state.dx) / r_old
        theta_relative_old = jnp.abs(angle_normalize(old_state.theta - positional_angle_old))

        # Landing conditions
        slow = v_total < params.landing_max_speed
        correct_position = delta_angle <= params.landing_position_tolerance
        correct_orientation = jnp.abs(theta_relative) <= params.landing_max_angle
        low_spin = jnp.abs(new_state.omega) <= params.landing_max_omega

        # Terminal rewards
        terminal_reward = jnp.array(0.0)
        sl_cp_co_ls = landed & slow & correct_position & correct_orientation & low_spin
        terminal_reward = jnp.where(sl_cp_co_ls, params.reward_sl_cp_co_ls, terminal_reward)
        sl_ncp_co_ls = landed & slow & ~correct_position & correct_orientation & low_spin
        terminal_reward = jnp.where(sl_ncp_co_ls, params.reward_sl_ncp_co_ls * jnp.exp(-delta_angle * 2.0), terminal_reward)
        sl_nco_nls = landed & slow & ~(correct_orientation & low_spin)
        terminal_reward = jnp.where(sl_nco_nls, 100.0 * jnp.exp(-delta_angle), terminal_reward)
        nsl_cp = landed & ~slow & correct_position
        terminal_reward = jnp.where(nsl_cp, params.reward_nsl_cp, terminal_reward)
        nsl_ncp = landed & ~slow & ~correct_position
        terminal_reward = jnp.where(nsl_ncp, params.reward_nsl_ncp, terminal_reward)
        terminal_reward = jnp.where(escaped, params.reward_out_of_bounds, terminal_reward)
        terminal_reward = jnp.where(timeout & ~landed & ~escaped, params.reward_timeout, terminal_reward)

        ### Non-terminal (shaping) rewards
        shaping_reward = jnp.array(0.0)

        # Reward for firing engine while pointed retrograde (opposite to velocity direction)
        # This encourages braking burns to slow down
        thrust_angle = new_state.theta + new_state.gimbal
        velocity_angle = jnp.arctan2(new_state.dy, new_state.dx)
        alignment = jnp.cos(thrust_angle - velocity_angle)  # 1 = prograde, -1 = retrograde
        retrograde_alignment = (1.0 - alignment) / 2.0  # 0 = prograde, 1 = retrograde
        throttle_used = new_state.throttle > 0.1
        velocity_factor = jnp.minimum(v_total / 10.0, 1.0)
        retrograde_burn_reward = params.reward_retrograde_burn * retrograde_alignment * throttle_used * velocity_factor
        shaping_reward += retrograde_burn_reward

        # Reward for decreasing tangential velocity (we want v_tangential -> 0)
        # Negative coefficient because we want to reward when (new - old) is negative
        tangential_velocity_change = jnp.abs(v_tangential) - jnp.abs(v_tangential_old)
        shaping_reward += -0.3 * tangential_velocity_change

        # Reward for decreasing altitude, but only when near the target landing spot
        # Weight by exp(-delta_angle) so we don't reward descending on the wrong side
        delta_angle_factor = jnp.exp(-delta_angle * 3)
        altitude_change = normalized_altitude - normalized_altitude_old
        shaping_reward += -1.0 * delta_angle_factor * altitude_change

        # Reward for improving orientation (theta_relative -> 0), but only when near landing
        # near_landing_factor is high only when both altitude and velocity are low
        near_landing_factor = jnp.exp(-normalized_altitude * 3.0) * jnp.exp(-v_total)
        theta_relative_change = jnp.abs(theta_relative) - jnp.abs(theta_relative_old)
        shaping_reward += -1.0 * near_landing_factor * theta_relative_change

        # Reward for decreasing angular velocity (omega -> 0), but only when near landing
        omega_change = jnp.abs(new_state.omega) - jnp.abs(old_state.omega)
        shaping_reward += -0.1 * near_landing_factor * omega_change

        # Bonus for being in the "landing zone" (low altitude AND near target)
        # This is a constant bonus per timestep, not a change-based reward
        low_altitude_factor = jnp.exp(-normalized_altitude * 2.0)
        near_target_factor = jnp.exp(-delta_angle * 2.0)
        landing_zone_bonus = 0.1 * low_altitude_factor * near_target_factor * near_target_factor
        shaping_reward += landing_zone_bonus

        # Reward for decreasing total velocity (we want v_total -> 0)
        # Only reward decreases, don't punish increases (use max with 0)
        velocity_change = v_total - v_total_old
        shaping_reward += -0.1 * jnp.maximum(0.0, -velocity_change)  # reward when velocity_change < 0

        # Punish high angular velocity (spinning too fast)
        high_omega = jnp.abs(new_state.omega) > 2.0
        angular_velocity_punishment = 0.1 * high_omega * (jnp.abs(new_state.omega) - 2.0)
        shaping_reward -= angular_velocity_punishment

        # Constant penalty for being far from target (angular distance)
        # This discourages taking the "long way around" the planet
        distance_from_target_penalty = 0.02 * delta_angle
        shaping_reward -= distance_from_target_penalty

        return jnp.where(is_terminal, terminal_reward, shaping_reward)

    @property
    def name(self) -> str:
        return "RocketLander"

    @property
    def num_actions(self) -> int:
        return 2

    def action_space(self, params: Optional[EnvParams] = None) -> spaces.Box:
        return spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=jnp.float32)

    def observation_space(self, params: Optional[EnvParams] = None) -> spaces.Box:
        low = jnp.array([-10.0, -jnp.pi, -50.0, -50.0, -jnp.pi, -10.0, 0.0, -0.5, 0.0])
        high = jnp.array([150.0, jnp.pi, 50.0, 50.0, jnp.pi, 10.0, 1.0, 0.5, 5000.0])
        return spaces.Box(low, high, shape=(9,), dtype=jnp.float32)

## 3. Visualization Utilities

In [ ]:
def visualize_trajectory(states: list, params: EnvParams, title: str = "Rocket Landing Trajectory"):
    """Visualize the rocket trajectory around the planet."""
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))

    # Draw planet
    theta = np.linspace(0, 2 * np.pi, 100)
    planet_x = params.planet_radius * np.cos(theta)
    planet_y = params.planet_radius * np.sin(theta)
    ax.fill(planet_x, planet_y, color='#3d5a80', alpha=0.8, label='Planet')
    ax.plot(planet_x, planet_y, color='#1d3557', linewidth=2)

    # Extract trajectory
    xs = np.array([float(s.x) for s in states])
    ys = np.array([float(s.y) for s in states])
    throttles = np.array([float(s.throttle) for s in states])

    # Draw trajectory colored by throttle
    points = np.array([xs, ys]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    norm = plt.Normalize(0, 1)
    lc = LineCollection(segments, cmap='YlOrRd', norm=norm, linewidth=2, alpha=0.8)
    lc.set_array(throttles[:-1])
    ax.add_collection(lc)
    cbar = plt.colorbar(lc, ax=ax, label='Throttle')
    cbar.ax.yaxis.set_tick_params(color='white')
    cbar.ax.yaxis.label.set_color('white')
    plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='white')

    # Mark start and end
    ax.scatter(xs[0], ys[0], s=100, c='green', marker='o', zorder=5, label='Start')
    ax.scatter(xs[-1], ys[-1], s=100, c='red', marker='x', zorder=5, label='End')

    # Draw target landing site
    target_angle = float(states[0].target_angle)
    target_x = params.planet_radius * np.cos(target_angle)
    target_y = params.planet_radius * np.sin(target_angle)
    ax.scatter(target_x, target_y, s=200, c='yellow', marker='*', zorder=5, edgecolors='black', label='Target')

    # Draw rocket orientation arrows
    n_arrows = min(30, len(states))
    arrow_indices = np.linspace(0, len(states) - 1, n_arrows, dtype=int)
    for idx in arrow_indices:
        s = states[idx]
        x, y = float(s.x), float(s.y)
        th = float(s.theta)
        dx = np.sin(th) * 8
        dy = np.cos(th) * 8
        ax.arrow(x, y, dx, dy, head_width=4, head_length=2, fc='#cccccc', ec='white', alpha=0.9)

    # Set view
    fov = params.init_max_orbit_radius * 1.2
    ax.set_xlim(-fov, fov)
    ax.set_ylim(-fov, fov)
    ax.set_aspect('equal')
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.set_title(title)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

    # Dark background
    ax.set_facecolor('#0d1b2a')
    fig.patch.set_facecolor('#0d1b2a')
    ax.tick_params(colors='white')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.title.set_color('white')
    ax.legend(facecolor='#1b263b', edgecolor='white', labelcolor='white')

    plt.tight_layout()
    plt.show()
    return fig

## 4. Neural Network Policy

In [ ]:
def initialize_mlp(layer_sizes, key: PRNGKey, scale: float = 1e-2):
    """Initialize MLP parameters."""
    keys = jr.split(key, 2 * len(layer_sizes))
    params = []
    for i in range(len(layer_sizes) - 1):
        input_size, output_size = layer_sizes[i], layer_sizes[i + 1]
        W = jr.normal(keys[2 * i], (input_size, output_size)) * scale
        b = jr.normal(keys[2 * i + 1], (output_size,)) * scale
        params.append((W, b))
    return params


def policy(params, x):
    """Gaussian policy network."""
    for (W, b) in params[:-1]:
        x = jnp.dot(x, W) + b
        x = jax.nn.relu(x)

    W, b = params[-1]
    output = jnp.dot(x, W) + b
    
    action_dim = output.shape[-1] // 2
    mean = jnp.tanh(output[:action_dim])
    log_std = jnp.clip(output[action_dim:], -2.0, 0.5)
    
    return mean, log_std


def get_action(params, x, key: PRNGKey):
    """Sample an action from the Gaussian policy."""
    mean, log_std = policy(params, x)
    std = jnp.exp(log_std)
    noise = jr.normal(key, shape=mean.shape)
    action = jnp.clip(mean + std * noise, -1.0, 1.0)
    return action, mean, log_std


def get_log_prob(params, x, action):
    """Compute log probability of action under the Gaussian policy."""
    mean, log_std = policy(params, x)
    std = jnp.exp(log_std)
    log_prob = -0.5 * jnp.sum(((action - mean) / std) ** 2 + 2 * log_std + jnp.log(2 * jnp.pi))
    return log_prob


@jax.jit
def update_delta(delta, grad_theta):
    """Update the parameter gradients."""
    return jax.tree.map(lambda x, y: x + y, delta, grad_theta), None

# 🚀 Rocket Landing with Reinforcement Learning

This notebook trains a neural network policy to land a rocket on a planet using REINFORCE.

**Features:**
- 2D physics simulation with gravity
- Rocket starts in orbit, must land at target location
- Continuous control: throttle and gimbal angle
- GPU-accelerated training with JAX

## Setup

First, let's check if we have GPU access and install dependencies.

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q jax[cuda12] flax optax gymnax matplotlib numpy

In [ ]:
# Verify JAX GPU setup
import jax
print(f"JAX version: {jax.__version__}")
print(f"Backend: {jax.default_backend()}")
print(f"Devices: {jax.devices()}")

if jax.default_backend() == 'gpu':
    print("\n✅ GPU acceleration enabled!")
else:
    print("\n⚠️ Running on CPU. Go to Runtime > Change runtime type > GPU")

## Environment Definition

The rocket environment simulates:
- A planet with gravitational pull
- A rocket that starts in orbit
- Target landing location on the planet surface
- Fuel consumption and thrust physics

In [ ]:
"""JAX implementation of rocket landing environment."""
from typing import Any, Optional, Tuple
import jax
import jax.numpy as jnp
from flax import struct
from gymnax.environments import environment, spaces

# GRAVITATIONAL_CONSTANT
G = 6.67430e-11

@struct.dataclass
class EnvState(environment.EnvState):
    """State of the rocket landing environment."""
    x: jnp.ndarray
    y: jnp.ndarray
    theta: jnp.ndarray
    dx: jnp.ndarray
    dy: jnp.ndarray
    omega: jnp.ndarray
    throttle: jnp.ndarray
    gimbal: jnp.ndarray
    fuel: jnp.ndarray
    target_angle: jnp.ndarray
    time: jnp.ndarray

@struct.dataclass
class EnvParams(environment.EnvParams):
    """Parameters for the rocket landing environment."""
    dt: float = 0.05
    planet_radius: float = 50.
    planet_mass: float = 5.0e14
    rocket_max_thrust: float = 38000.0
    rocket_max_gimbal: float = 0.3
    rocket_mass: float = 2600.0
    rocket_moment_of_inertia: float = 250.0
    rocket_initial_fuel: float = 5000.0
    rocket_fuel_consumption_rate: float = 100.
    init_min_orbit_radius: float = 80.0
    init_max_orbit_radius: float = 150.0
    init_orbit_velocity_noise: float = 0.1
    max_steps_in_episode: int = 1500
    landing_max_speed: float = 2.0
    landing_max_angle: float = 0.3
    landing_max_omega: float = 0.5
    landing_position_tolerance: float = jnp.pi/30
    # Terminal rewards
    reward_sl_cp_co_ls: float = 1000.0
    reward_nsl_cp: float = 200.0
    reward_nsl_ncp: float = 100.0
    reward_sl_ncp_co_ls: float = 500.0
    reward_out_of_bounds: float = -1000.0
    reward_timeout: float = -2000.0
    reward_retrograde_burn: float = 0.02

def angle_normalize(x):
    """Normalize the angle - radians."""
    return ((x + jnp.pi) % (2 * jnp.pi)) - jnp.pi

class RocketLander(environment.Environment[EnvState, EnvParams]):
    """JAX/Gymnax implementation of a 2D rocket landing environment."""

    def __init__(self):
        super().__init__()

    @property
    def default_params(self) -> EnvParams:
        return EnvParams()

    def step_env(self, key, state, action, params):
        """Integrate physics for one timestep."""
        r = jnp.sqrt(state.x**2 + state.y**2)
        a_gravity_magnitude = G * params.planet_mass / r**2
        a_g_x = -a_gravity_magnitude * state.x / r
        a_g_y = -a_gravity_magnitude * state.y / r

        throttle = jnp.clip(action[0], -1.0, 1.0)
        gimbal = jnp.clip(action[1], -1.0, 1.0)
        throttle = (throttle + 1.0) / 2.0
        gimbal *= params.rocket_max_gimbal
        throttle = jnp.minimum(throttle, state.fuel / (params.rocket_fuel_consumption_rate * params.dt))

        thrust_angle = state.theta + gimbal
        F_thrust = throttle * params.rocket_max_thrust
        a_t_x = F_thrust * jnp.sin(thrust_angle) / params.rocket_mass
        a_t_y = F_thrust * jnp.cos(thrust_angle) / params.rocket_mass
        a_omega = F_thrust * jnp.sin(gimbal) / params.rocket_moment_of_inertia

        a_x = a_g_x + a_t_x
        a_y = a_g_y + a_t_y
        dx = state.dx + a_x * params.dt
        dy = state.dy + a_y * params.dt
        omega = state.omega + a_omega * params.dt
        x = state.x + dx * params.dt
        y = state.y + dy * params.dt
        theta = state.theta + omega * params.dt
        fuel = jnp.maximum(0.0, state.fuel - throttle * params.rocket_fuel_consumption_rate * params.dt)

        new_state = EnvState(
            x=x, y=y, theta=theta, dx=dx, dy=dy, omega=omega,
            throttle=throttle, gimbal=gimbal, fuel=fuel,
            target_angle=state.target_angle, time=state.time + params.dt
        )
        done = self.is_terminal(new_state, params)
        reward = self._compute_reward(state, new_state, params)
        key, noise_key = jax.random.split(key)
        obs = self.get_obs(new_state, params, noise_key)
        return obs, new_state, reward, done, {}

    def reset_env(self, key, params):
        """Reset environment by sampling initial orbit."""
        key, angle_key, orbit_radius_key, orbit_vel_noise_key, target_angle_key = jax.random.split(key, 5)
        angle_init = jax.random.uniform(angle_key, minval=0.0, maxval=2.0 * jnp.pi)
        orbit_radius_init = jax.random.uniform(orbit_radius_key, minval=params.init_min_orbit_radius, maxval=params.init_max_orbit_radius)
        x_init = orbit_radius_init * jnp.cos(angle_init)
        y_init = orbit_radius_init * jnp.sin(angle_init)
        orbit_velocity_magnitude_init = jnp.sqrt(G * params.planet_mass / orbit_radius_init)
        velocity_noise = params.init_orbit_velocity_noise * (2.0 * jax.random.uniform(orbit_vel_noise_key) - 1.0)
        orbit_velocity_magnitude_init *= (1.0 + velocity_noise)
        dx_init = -orbit_velocity_magnitude_init * jnp.sin(angle_init)
        dy_init = orbit_velocity_magnitude_init * jnp.cos(angle_init)
        theta_init = angle_init + (3/2) * jnp.pi
        target_angle = jax.random.uniform(target_angle_key, minval=0.0, maxval=2.0 * jnp.pi)

        state = EnvState(
            x=x_init, y=y_init, theta=theta_init, dx=dx_init, dy=dy_init,
            omega=jnp.array(0.0), throttle=jnp.array(0.0), gimbal=jnp.array(0.0),
            fuel=jnp.array(params.rocket_initial_fuel), target_angle=target_angle, time=jnp.array(0)
        )
        return self.get_obs(state, params, key), state

    def get_obs(self, state, params=None, key=None):
        """Get observation vector."""
        if params is None:
            params = self.default_params
        r = jnp.sqrt(state.x ** 2 + state.y ** 2)
        altitude = r - params.planet_radius
        positional_angle = jnp.arctan2(state.y, state.x)
        delta_angle = angle_normalize(state.target_angle - positional_angle)
        radial_vel = (state.x * state.dx + state.y * state.dy) / r
        tangential_vel = (state.x * state.dy - state.y * state.dx) / r
        theta_relative = angle_normalize(state.theta - positional_angle)
        return jnp.array([altitude, delta_angle, radial_vel, tangential_vel, theta_relative, state.omega, state.throttle, state.gimbal, state.fuel])

    def is_terminal(self, state, params):
        """Check whether state is terminal."""
        max_time = params.max_steps_in_episode * params.dt
        timeout = state.time >= max_time
        r = jnp.sqrt(state.x ** 2 + state.y ** 2)
        landed = r <= params.planet_radius
        escaped = r > params.planet_radius * 4
        return jnp.array(landed | timeout | escaped)

    def _compute_reward(self, old_state, new_state, params):
        """Compute reward with terminal rewards REPLACING shaping rewards."""
        r = jnp.sqrt(new_state.x ** 2 + new_state.y ** 2)
        altitude = r - params.planet_radius
        normalized_altitude = jnp.clip(altitude / (params.init_max_orbit_radius - params.planet_radius), 0.0, 2.0)
        landed = r <= params.planet_radius
        escaped = r > params.planet_radius * 4
        max_time = params.max_steps_in_episode * params.dt
        timeout = new_state.time >= max_time
        is_terminal = landed | escaped | timeout

        positional_angle = jnp.arctan2(new_state.y, new_state.x)
        delta_angle = jnp.abs(angle_normalize(new_state.target_angle - positional_angle))
        v_total = jnp.sqrt(new_state.dx ** 2 + new_state.dy ** 2)
        v_tangential = (new_state.x * new_state.dy - new_state.y * new_state.dx) / r
        theta_relative = angle_normalize(new_state.theta - positional_angle)

        r_old = jnp.sqrt(old_state.x ** 2 + old_state.y ** 2)
        normalized_altitude_old = jnp.clip((r_old - params.planet_radius) / (params.init_max_orbit_radius - params.planet_radius), 0.0, 2.0)
        positional_angle_old = jnp.arctan2(old_state.y, old_state.x)
        delta_angle_old = jnp.abs(angle_normalize(old_state.target_angle - positional_angle_old))
        v_total_old = jnp.sqrt(old_state.dx ** 2 + old_state.dy ** 2)
        v_tangential_old = (old_state.x * old_state.dy - old_state.y * old_state.dx) / r_old
        theta_relative_old = jnp.abs(angle_normalize(old_state.theta - positional_angle_old))

        slow = v_total < params.landing_max_speed
        correct_position = delta_angle <= params.landing_position_tolerance
        correct_orientation = jnp.abs(theta_relative) <= params.landing_max_angle
        low_spin = jnp.abs(new_state.omega) <= params.landing_max_omega

        # Terminal rewards
        terminal_reward = jnp.array(0.0)
        sl_cp_co_ls = landed & slow & correct_position & correct_orientation & low_spin
        terminal_reward = jnp.where(sl_cp_co_ls, params.reward_sl_cp_co_ls, terminal_reward)
        sl_ncp_co_ls = landed & slow & ~correct_position & correct_orientation & low_spin
        terminal_reward = jnp.where(sl_ncp_co_ls, params.reward_sl_ncp_co_ls * jnp.exp(-delta_angle * 2.0), terminal_reward)
        sl_nco_nls = landed & slow & ~(correct_orientation & low_spin)
        terminal_reward = jnp.where(sl_nco_nls, 100.0 * jnp.exp(-delta_angle), terminal_reward)
        nsl_cp = landed & ~slow & correct_position
        terminal_reward = jnp.where(nsl_cp, params.reward_nsl_cp, terminal_reward)
        nsl_ncp = landed & ~slow & ~correct_position
        terminal_reward = jnp.where(nsl_ncp, params.reward_nsl_ncp, terminal_reward)
        terminal_reward = jnp.where(escaped, params.reward_out_of_bounds, terminal_reward)
        terminal_reward = jnp.where(timeout & ~landed & ~escaped, params.reward_timeout, terminal_reward)

        ### Non-terminal (shaping) rewards
        shaping_reward = jnp.array(0.0)

        # Reward for firing engine while pointed retrograde (opposite to velocity direction)
        # This encourages braking burns to slow down
        thrust_angle = new_state.theta + new_state.gimbal
        velocity_angle = jnp.arctan2(new_state.dy, new_state.dx)
        alignment = jnp.cos(thrust_angle - velocity_angle)  # 1 = prograde, -1 = retrograde
        retrograde_alignment = (1.0 - alignment) / 2.0  # 0 = prograde, 1 = retrograde
        throttle_used = new_state.throttle > 0.1
        velocity_factor = jnp.minimum(v_total / 10.0, 1.0)
        retrograde_burn_reward = params.reward_retrograde_burn * retrograde_alignment * throttle_used * velocity_factor
        shaping_reward += retrograde_burn_reward

        # Reward for decreasing tangential velocity (we want v_tangential -> 0)
        # Negative coefficient because we want to reward when (new - old) is negative
        tangential_velocity_change = jnp.abs(v_tangential) - jnp.abs(v_tangential_old)
        shaping_reward += -0.3 * tangential_velocity_change

        # Reward for decreasing altitude, but only when near the target landing spot
        # Weight by exp(-delta_angle) so we don't reward descending on the wrong side
        delta_angle_factor = jnp.exp(-delta_angle * 3)
        altitude_change = normalized_altitude - normalized_altitude_old
        shaping_reward += -1.0 * delta_angle_factor * altitude_change

        # Reward for improving orientation (theta_relative -> 0), but only when near landing
        # near_landing_factor is high only when both altitude and velocity are low
        near_landing_factor = jnp.exp(-normalized_altitude * 3.0) * jnp.exp(-v_total)
        theta_relative_change = jnp.abs(theta_relative) - jnp.abs(theta_relative_old)
        shaping_reward += -1.0 * near_landing_factor * theta_relative_change

        # Reward for decreasing angular velocity (omega -> 0), but only when near landing
        omega_change = jnp.abs(new_state.omega) - jnp.abs(old_state.omega)
        shaping_reward += -0.1 * near_landing_factor * omega_change

        # Bonus for being in the "landing zone" (low altitude AND near target)
        # This is a constant bonus per timestep, not a change-based reward
        low_altitude_factor = jnp.exp(-normalized_altitude * 2.0)
        near_target_factor = jnp.exp(-delta_angle * 2.0)
        landing_zone_bonus = 0.1 * low_altitude_factor * near_target_factor * near_target_factor
        shaping_reward += landing_zone_bonus

        # Reward for decreasing total velocity (we want v_total -> 0)
        # Only reward decreases, don't punish increases (use max with 0)
        velocity_change = v_total - v_total_old
        shaping_reward += -0.1 * jnp.maximum(0.0, -velocity_change)  # reward when velocity_change < 0

        # Punish high angular velocity (spinning too fast)
        high_omega = jnp.abs(new_state.omega) > 2.0
        angular_velocity_punishment = 0.1 * high_omega * (jnp.abs(new_state.omega) - 2.0)
        shaping_reward -= angular_velocity_punishment

        # Constant penalty for being far from target (angular distance)
        # This discourages taking the "long way around" the planet
        distance_from_target_penalty = 0.02 * delta_angle
        shaping_reward -= distance_from_target_penalty

        return jnp.where(is_terminal, terminal_reward, shaping_reward)

    @property
    def name(self):
        return "RocketLander"

    @property
    def num_actions(self):
        return 2

    def action_space(self, params=None):
        return spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=jnp.float32)

    def observation_space(self, params=None):
        low = jnp.array([-10.0, -jnp.pi, -50.0, -50.0, -jnp.pi, -10.0, 0.0, -0.5, 0.0])
        high = jnp.array([150.0, jnp.pi, 50.0, 50.0, jnp.pi, 10.0, 1.0, 0.5, 5000.0])
        return spaces.Box(low, high, shape=(9,), dtype=jnp.float32)

print("✅ Environment defined!")

## Visualization Utilities

In [ ]:
"""Visualization utilities."""
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection

def visualize_trajectory(states, params, title="Rocket Landing Trajectory", save_path=None):
    """Visualize the rocket trajectory around the planet."""
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    # Draw planet
    theta = np.linspace(0, 2 * np.pi, 100)
    planet_x = params.planet_radius * np.cos(theta)
    planet_y = params.planet_radius * np.sin(theta)
    ax.fill(planet_x, planet_y, color='#3d5a80', alpha=0.8, label='Planet')
    ax.plot(planet_x, planet_y, color='#1d3557', linewidth=2)

    # Extract trajectory
    xs = np.array([float(s.x) for s in states])
    ys = np.array([float(s.y) for s in states])
    throttles = np.array([float(s.throttle) for s in states])

    # Draw trajectory colored by throttle
    points = np.array([xs, ys]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    norm = plt.Normalize(0, 1)
    lc = LineCollection(segments, cmap='YlOrRd', norm=norm, linewidth=2, alpha=0.8)
    lc.set_array(throttles[:-1])
    ax.add_collection(lc)
    cbar = plt.colorbar(lc, ax=ax, label='Throttle')
    cbar.ax.yaxis.set_tick_params(color='white')
    cbar.ax.yaxis.label.set_color('white')
    plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='white')

    # Markers
    ax.scatter(xs[0], ys[0], s=100, c='green', marker='o', zorder=5, label='Start')
    ax.scatter(xs[-1], ys[-1], s=100, c='red', marker='x', zorder=5, label='End')

    # Target
    target_angle = float(states[0].target_angle)
    target_x = params.planet_radius * np.cos(target_angle)
    target_y = params.planet_radius * np.sin(target_angle)
    ax.scatter(target_x, target_y, s=200, c='yellow', marker='*', zorder=5, edgecolors='black', label='Target')

    # Rocket orientation arrows
    n_arrows = min(30, len(states))
    arrow_indices = np.linspace(0, len(states) - 1, n_arrows, dtype=int)
    for idx in arrow_indices:
        s = states[idx]
        x, y = float(s.x), float(s.y)
        theta = float(s.theta)
        dx = np.sin(theta) * 8
        dy = np.cos(theta) * 8
        ax.arrow(x, y, dx, dy, head_width=4, head_length=2, fc='#cccccc', ec='white', alpha=0.9)

    # Styling
    fov = params.init_max_orbit_radius * 1.2
    ax.set_xlim(-fov, fov)
    ax.set_ylim(-fov, fov)
    ax.set_aspect('equal')
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.set_title(title)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_facecolor('#0d1b2a')
    fig.patch.set_facecolor('#0d1b2a')
    ax.tick_params(colors='white')
    ax.xaxis.label.set_color('white')
    ax.yaxis.label.set_color('white')
    ax.title.set_color('white')
    ax.legend(facecolor='#1b263b', edgecolor='white', labelcolor='white')

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=150, facecolor=fig.get_facecolor())
        plt.close()
    else:
        plt.show()
    return fig

print("✅ Visualization utilities defined!")

## Neural Network Policy and Training

We use REINFORCE with a Gaussian policy to train the rocket landing agent.

In [ ]:
"""Training components."""
import jax.random as jr
from jax.random import PRNGKey
import optax

# Create the environment
env = RocketLander()
env_params = env.default_params

def initialize_mlp(layer_sizes, key, scale=1e-2):
    """Initialize MLP parameters."""
    keys = jr.split(key, 2 * len(layer_sizes))
    params = []
    for i in range(len(layer_sizes) - 1):
        input_size, output_size = layer_sizes[i], layer_sizes[i + 1]
        W = jr.normal(keys[2 * i], (input_size, output_size)) * scale
        b = jr.normal(keys[2 * i + 1], (output_size,)) * scale
        params.append((W, b))
    return params

def policy(params, x):
    """Gaussian policy network."""
    for (W, b) in params[:-1]:
        x = jnp.dot(x, W) + b
        x = jax.nn.relu(x)
    W, b = params[-1]
    output = jnp.dot(x, W) + b
    action_dim = output.shape[-1] // 2
    mean = jnp.tanh(output[:action_dim])
    log_std = jnp.clip(output[action_dim:], -2.0, 0.5)
    return mean, log_std

def get_action(params, x, key):
    """Sample an action from the Gaussian policy."""
    mean, log_std = policy(params, x)
    std = jnp.exp(log_std)
    noise = jr.normal(key, shape=mean.shape)
    action = jnp.clip(mean + std * noise, -1.0, 1.0)
    return action, mean, log_std

def get_log_prob(params, x, action):
    """Compute log probability of action under the Gaussian policy."""
    mean, log_std = policy(params, x)
    std = jnp.exp(log_std)
    return -0.5 * jnp.sum(((action - mean) / std) ** 2 + 2 * log_std + jnp.log(2 * jnp.pi))

@jax.jit
def update_delta(delta, grad_theta):
    """Update the parameter gradients."""
    return jax.tree.map(lambda x, y: x + y, delta, grad_theta), None

def rollout(params, env_params, rng_input, steps_in_episode):
    """Rollout an episode using the policy."""
    rng_reset, rng_episode = jr.split(rng_input)
    obs, state = env.reset_env(rng_reset, env_params)

    def policy_step(state_input, tmp):
        obs, state, rng = state_input
        rng, rng_action, rng_step = jr.split(rng, 3)
        action, mean, log_std = get_action(params, obs, rng_action)
        next_obs, next_state, reward, done, _ = env.step_env(rng_step, state, action, env_params)
        carry = [next_obs, next_state, rng]
        return carry, [obs, state, action, reward, next_obs, done]

    _, scan_out = jax.lax.scan(policy_step, [obs, state, rng_episode], (), length=steps_in_episode)
    return scan_out

jit_rollout = jax.jit(rollout, static_argnums=3)

def loss_REINFORCE(params, obs, action, reward, baseline, gamma=0.99):
    """Compute REINFORCE loss with baseline."""
    def trajectory_gradients(reward, obs, action, baseline, delta):
        G_init = 0.0
        def step(carry, variables):
            G, delta = carry
            r, obs, action, b = variables
            G = gamma * G + r
            advantage = G - b
            def neg_log_prob(params):
                return -get_log_prob(params, obs, action)
            grad_delta = jax.grad(neg_log_prob)(params)
            grad_delta = jax.tree.map(lambda gd: gd * advantage, grad_delta)
            delta, _ = update_delta(delta, grad_delta)
            return (G, delta), G
        variables = (reward[::-1], obs[::-1], action[::-1], baseline[::-1])
        (_, delta), Gt = jax.lax.scan(step, (G_init, delta), variables)
        return delta, Gt

    parallel_trajectory_gradients = jax.vmap(trajectory_gradients, in_axes=(0, 0, 0, None, None))
    delta = jax.tree.map(lambda t: jnp.zeros(t.shape), params)
    deltas, Gs = parallel_trajectory_gradients(reward, obs, action, baseline, delta)
    delta, _ = jax.lax.scan(update_delta, delta, deltas)
    return delta, jnp.array(Gs)

loss_REINFORCE = jax.jit(loss_REINFORCE)

def mean_baseline(Gs):
    """Compute constant baseline as mean of discounted returns."""
    T = Gs.shape[1]
    return jnp.ones((T,)) * jnp.mean(Gs)

print("✅ Training components defined!")

## Training Function

In [ ]:
def train(
    num_iters=5000,
    steps_in_episode=1500,
    lr=0.0002,
    gamma=0.995,
    n_batches=128,
    hidden_size=256,
    n_hidden_layers=3,
    visualize_every=500,
    seed=42,
    grad_clip=0.5,
):
    """Train the rocket landing policy using REINFORCE."""
    key = PRNGKey(seed)
    
    # Network architecture
    obs_dim = 9
    action_dim = 2
    hidden_layers = tuple([hidden_size] * n_hidden_layers)
    layer_sizes = (obs_dim,) + hidden_layers + (2 * action_dim,)
    
    # Initialize
    key, subkey = jr.split(key)
    params = initialize_mlp(layer_sizes, key=subkey)
    optim = optax.chain(optax.clip_by_global_norm(grad_clip), optax.adam(learning_rate=lr))
    opt_state = optim.init(params)
    Gs = jnp.zeros((n_batches, steps_in_episode))
    
    key, subkey = jr.split(key)
    iter_keys = jr.split(subkey, num_iters)
    parallel_rollout = jax.vmap(rollout, in_axes=(None, None, 0, None))
    
    def step(carry, key):
        params, opt_state, Gs = carry
        keys = jr.split(key, n_batches)
        obs, state, action, reward, next_obs, done = parallel_rollout(params, env_params, keys, steps_in_episode)
        baseline = mean_baseline(Gs)
        delta, Gs_new = loss_REINFORCE(params, obs, action, reward, baseline, gamma)
        updates, opt_state = optim.update(delta, opt_state, params)
        new_params = optax.apply_updates(params, updates)
        mean_reward = jnp.mean(jnp.sum(reward, axis=-1))
        return (new_params, opt_state, Gs_new), mean_reward

    step = jax.jit(step)
    
    history = []
    current_params = params
    current_opt_state = opt_state
    current_Gs = Gs
    
    print(f"🚀 Starting training for {num_iters} iterations...")
    print(f"   Network: {layer_sizes}")
    print(f"   Batch size: {n_batches}, Steps: {steps_in_episode}")
    print(f"   LR: {lr}, Gamma: {gamma}")
    print()
    
    for i in range(num_iters):
        (current_params, current_opt_state, current_Gs), mean_reward = step(
            (current_params, current_opt_state, current_Gs), iter_keys[i]
        )
        history.append(float(mean_reward))
        
        if (i + 1) % 100 == 0:
            avg_reward = jnp.mean(jnp.array(history[-100:]))
            print(f"Iteration {i + 1}/{num_iters} | Avg Reward (last 100): {avg_reward:.2f}")
        
        if (i + 1) % visualize_every == 0:
            print(f"\n📊 Visualizing at iteration {i + 1}...")
            key, vis_key = jr.split(key)
            visualize_policy(current_params, vis_key, title=f"Iteration {i + 1}")
    
    print("\n✅ Training complete!")
    key, vis_key = jr.split(key)
    visualize_policy(current_params, vis_key, title="Final Policy")
    
    # Plot training curve
    plt.figure(figsize=(12, 4))
    plt.plot(history, alpha=0.3, label='Episode Reward')
    window = min(100, len(history) // 10)
    if window > 1:
        smoothed = jnp.convolve(jnp.array(history), jnp.ones(window) / window, mode='valid')
        plt.plot(range(window - 1, len(history)), smoothed, label=f'Smoothed (window={window})')
    plt.xlabel('Iteration')
    plt.ylabel('Mean Episode Reward')
    plt.title('Training Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return current_params, history


def visualize_policy(params, key, title="Policy Trajectory"):
    """Run a single episode and visualize the trajectory."""
    obs, state, action, reward, next_obs, done = rollout(params, env_params, key, env_params.max_steps_in_episode)
    states = []
    for i in range(len(state.x)):
        s = EnvState(
            x=state.x[i], y=state.y[i], theta=state.theta[i],
            dx=state.dx[i], dy=state.dy[i], omega=state.omega[i],
            throttle=state.throttle[i], gimbal=state.gimbal[i],
            fuel=state.fuel[i], target_angle=state.target_angle[i], time=state.time[i],
        )
        states.append(s)
        if done[i]:
            break
    
    # Only sum rewards up to the terminal step (after that, rewards are garbage due to r=0)
    episode_length = len(states)
    total_reward = float(jnp.sum(reward[:episode_length]))
    print(f"  Total reward: {total_reward:.2f}, Steps: {episode_length}")
    visualize_trajectory(states, env_params, title=f"{title} (R={total_reward:.1f})")

print("✅ Training function defined!")

## 🎮 Run Training!

Adjust the parameters below and run training. With GPU, this should be quite fast!

In [ ]:
# Train the agent!
# Adjust num_iters for longer/shorter training

params, history = train(
    num_iters=10000,          # Number of training iterations
    steps_in_episode=1500,    # Max steps per episode
    lr=0.0002,                # Learning rate
    gamma=0.995,              # Discount factor
    n_batches=128,            # Parallel episodes per iteration
    hidden_size=256,          # Hidden layer size
    n_hidden_layers=3,        # Number of hidden layers
    visualize_every=1000,     # Visualize every N iterations
    seed=42,
    grad_clip=0.5,
)

## Test the Trained Policy

Run multiple episodes to see how the trained agent performs.

In [ ]:
# Test the trained policy multiple times
key = jr.PRNGKey(123)

for i in range(5):
    key, subkey = jr.split(key)
    print(f"\n--- Episode {i+1} ---")
    visualize_policy(params, subkey, title=f"Test Episode {i+1}")

## Save/Load Model

You can save the trained parameters to Google Drive.

In [ ]:
# Save model (optional)
import pickle

# Flatten params for saving
params_np = jax.tree.map(lambda x: np.array(x), params)

with open('rocket_lander_params.pkl', 'wb') as f:
    pickle.dump(params_np, f)

print("Model saved to rocket_lander_params.pkl")

# To load:
# with open('rocket_lander_params.pkl', 'rb') as f:
#     params_loaded = pickle.load(f)
# params_loaded = jax.tree.map(lambda x: jnp.array(x), params_loaded)